# HDF5 in Julia

So last summer, my program was producing three dimensional data, and I needed a way to export and save that data from my C++ program.  Simple ASCII files, my default method, no longer covered my needs.  Of course, I wasn't the first person to encounter this problem, so I discovered the HDF5 standard.  

Instead of storing data in a human readable format like ASCII, the Hierarchical Data Format, HDF, stores data in binary format.  This preserves the shape of the data in the computer and keeps it at its minimum size.  WOHOO!!

Sadly, the syntax for HDF5 in C++ and Fortran is just as bad as FFTW or OpenBLAS.  But happily, just like FFTW and OpenBLAS, HDF5 has wonderful syntax in Julia, Python, and MATLAB, amoung others.

So how does it work? 

We don't just print a single variable.  Each HDF5 file is like its own file system.  In my home directory, I have my documents folder, my programming folder, my pictures, configuration files,... and inside each folder I can have subfolders or files.  

The same is true for an HDF5 file.  We have the root, and then we have groups and subgroups.  A group is like a folder.  Then we can have datasets. Datasets are objects that hold data (files).  


## Installing the Package

While adding Julia's HDF5 package will hopefully add the HDF5 library as well, additional steps may be required. [Anaconda](https://www.anaconda.com/) can provide the system binaries.  After adding HDF5.jl, Julia may need to build it.  At the REPL, this can be done in package mode by:

    (M4) pkg> build HDF5
    
or more with `Pkg` itself

    using Pkg
    Pkg.build("HDF5")


In [1]:
using HDF5;

# Hello World 
First let's open a file and then write some data to it.

We can open a file in three ways:

|Symbol| Meaning|
|-----| ------|
|"w"| Write.  Will overwrite anything already there.|
|"r" |Ready-only.|
|"r+"| Read-write. Preserving existing contents. | 

If we open with this syntax, we have to always remember to close it with `close()`

In [2]:
fid=h5open("test.h5","w")

fid["string"]="Hello World"

close(fid)

### Navigating a File

Now let's see if we were successful by reading.  Instead of reading the dataset, we are going to checkout the structure of the file first.  

`names(fid)` tells us what is inside the location `fid`.  

Both these functions help you find your way around a file.

In [3]:
fid=h5open("test.h5","r")

println("names \n",names(fid))

close(fid)

names 
["string"]


### Reading Data
Now when we are reading data, we need to know the difference between dataset and the data the dataset contains. 

Look at the below example

In [4]:
fid=h5open("test.h5","r")

dset=fid["string"]
println("the dataset: \t", typeof(dset))

data=read(dset)
println("the string: \t", typeof(data),"\t",data)

data2=read(fid,"string")
println("read another way: \t", typeof(data2),"\t",data2)

close(fid)

the dataset: 	HDF5Dataset
the string: 	String	Hello World
read another way: 	String	Hello World


A dataset is like the filename "fairytale.txt", so we then need to read the file to get "Once upon a time ...".

### Groups
So I've talked about groups, but we haven't done anything with them yet. So let's make some groups!

Here we use `g_create` to create two groups, one inside the other.  For the subgroup, it's parent is `g`, so we have to create it at location `g`.  Just like in a filesystem, it's name/ path is nested within its parent's path.

In [5]:
fid=h5open("test.h5","w")

g=g_create(fid,"mygroup");
h=g_create(g,"mysubgroup");

println("\n path of h:  ",name(h))

close(fid)


 path of h:  /mygroup/mysubgroup


### Attributes

Say in a file I want to include the information that I ran the simulation with 100 sites, at 1 Kelvin, for 100,000 timesteps.  Instead of creating new datasets for each of these individual numbers, I can create attributes and tie them to either a group or a dataset.

In [6]:
fid=h5open("test.h5","w")

fid["data"]=randn(3,3);

attrs(fid["data"])["Temp"]="1";
attrs(fid["data"])["N Sites"]="100";

close(fid)

fid=h5open("test.h5","r")

dset=fid["data"];

println("typeof attrs: \t", typeof(attrs(dset)))
println("Temp: \t",read( attrs(dset),"Temp"  ))
println("N Sites: \t",read(  attrs(dset),"N Sites"  ))

close(fid)

typeof attrs: 	HDF5.HDF5Attributes
Temp: 	1
N Sites: 	100


### Final Tips

Before diving in to learn how to use this, think about whether you need it or not.  How large and complex is your data?  Is it worth the time to learn?  While the syntax might be relatively simple in Julia, ASCII files are still much easier to deal with.  

If you are going to play around or use this format, I recommend getting an HDF viewer, like https://www.hdfgroup.org/products/java/release/download.html While you can have much more control via code, sometimes it is just that much simpler to check everything is working via a GUI.   


For more information, checkout the Package page at https://github.com/JuliaLang/HDF5.jl or the HDFGroup page at https://www.hdfgroup.org/

I just showed some of the functionality in test cases.  As much control as you want, you can get; you might just have to work a bit for it.